In [1]:
#구글드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/2030런닝중 - 정형프로젝트/데이터셋

/content/drive/.shortcut-targets-by-id/1nyTaoNXGEUbaxGyohoBtcBoyRV-posbk/2030런닝중 - 정형프로젝트/데이터셋


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import datetime
sns.set_style("whitegrid")
import scipy.stats as scs
from sklearn.feature_selection import SelectKBest, chi2
from scipy.stats import chi2_contingency
from sklearn.preprocessing import OrdinalEncoder

In [4]:
data_2021 = pd.read_csv('5월_청년층부가조사/csv/2021_5월_청년층부가조사_20251222_04501.csv', encoding='cp949')
data_2022 = pd.read_csv('5월_청년층부가조사/csv/2022_5월_청년층부가조사_20251222_04501.csv', encoding='cp949')
data_2023 = pd.read_csv('5월_청년층부가조사/csv/2023_5월_청년층부가조사_20251222_04501.csv', encoding='cp949')
data_2024 = pd.read_csv('5월_청년층부가조사/csv/2024_5월_청년층부가조사_20251222_04501.csv', encoding='cp949')
data_2025 = pd.read_csv('5월_청년층부가조사/csv/2025_5월_청년층부가조사_20251222_04501.csv', encoding='cp949')
data_2021.head()

,조사연월,동부읍면부코드,가구주관계코드,성별코드,출생연도,교육정도_학력구분코드,교육정도_계열코드,교육정도_수학구분코드,졸업연도,혼인상태코드,...,첫직장_월평균급여액,첫직장취업연월,첫직장이직연월,첫직장퇴직사유코드,첫직장업종코드,첫직장_근로형태코드,최근직장_취업경로코드,최근직장_전공관련성코드,미취업_퇴직후미취업기간구분코드,미취업_주요활동상태코드
0,202105,1,1,1,1924,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
1,202105,1,1,1,1925,1,0,1,0,3,...,0,0,0,0,0,0,0,0,0,0
2,202105,1,1,1,1925,1,0,3,0,2,...,0,0,0,0,0,0,0,0,0,0
3,202105,1,1,1,1925,2,0,1,0,3,...,0,0,0,0,0,0,0,0,0,0
4,202105,1,1,1,1926,1,0,1,0,2,...,0,0,0,0,0,0,0,0,0,0


In [5]:
#data_2025에 없는 열 확인하기
for col in data_2023.columns:
    if col not in data_2025.columns:
        print(col)

현재일관련사항_10차산업대분류코드
현재일관련사항_7차직업대분류코드
이전직장_1자리_10차산업대분류코드
이전직장_1자리_7차직업대분류코드


In [6]:
#데이터 취합
datas_5 = pd.concat([data_2021, data_2022, data_2023, data_2024, data_2025])

In [7]:
datas_2030 = datas_5[(20 <= datas_5['만연령']) & (datas_5['만연령'] <= 38)]

In [8]:
datas_final = datas_2030[[
    '성별코드',
    '만연령',
    '연령계층코드',
    '혼인상태코드',
    '가구주관계코드',
    '교육정도_학력구분코드',
    '교육정도_계열코드',
    '가장최근학교_년제수',
    '가장최근학교_졸업및중퇴연월',
    '졸업연도',
    '가장최근학교_휴학경험유무',
    '가장최근학교_휴학기간년수',
    '직업교육및직장체험_직업교육수혜구분코드',
    '직업교육및직장체험_취업관련시험준비여부',
    '직업교육및직장체험_취업관련시험준비분야코드',
    '직업교육및직장체험_직장체험주요형태코드',
    '직업교육및직장체험_재학휴학중직장체험여부',
    '직업교육및직장체험_재학휴학중직장체험기간코드',
    '구직사항_구직활동기간',
    '구직사항_주요구직방법1코드',
    '구직사항_주요구직경로1코드'
    ]]

In [46]:
ordinal_cols = ['교육정도_학력구분코드', '연령계층코드']
numeric_cols = ['만연령', '구직사항_구직활동기간']

onehot_cols = [
    c for c in datas_final.columns
    if c not in ordinal_cols + numeric_cols
    and datas_final[c].nunique() <= 20
]

X = pd.get_dummies(
    datas_final,
    columns=onehot_cols,
    drop_first=True
)

datas_2030['이전직장사항_전직유무'] = datas_2030['이전직장사항_전직유무'].replace(2, 0)
y = datas_2030['이전직장사항_전직유무']


/tmp/ipython-input-1477575240.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datas_2030['이전직장사항_전직유무'] = datas_2030['이전직장사항_전직유무'].replace(2, 0)


In [10]:
!pip install catboost

In [47]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

# 1) train/test split
X_train_f, X_test_f, y_train_f, y_test_f = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

In [11]:
# 2) CatBoost with best params
cat = CatBoostClassifier(
    learning_rate=0.1,
    l2_leaf_reg=3,
    iterations=800,
    depth=10,
    loss_function="Logloss",
    eval_metric="AUC",
    bootstrap_type="Bernoulli",
    subsample=0.8,
    random_state=42,
    random_strength=1,
    scale_pos_weight=0.5, #0.5
    verbose=0
)

# 3) fit
cat.fit(X_train_f, y_train_F)

# 4) 기본 accuracy (참고용)
acc = cat.score(X_test_f, y_test_f)
print(f"Accuracy: {acc:.4f}")

Accuracy: 0.8214


In [12]:
from sklearn.metrics import classification_report, confusion_matrix
y_pred = cat.predict(X_test_f)

print("\nClassification Report:\n", classification_report(y_test, y_pred))


Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.97      0.89      9009
           1       0.76      0.32      0.45      2673

    accuracy                           0.82     11682
   macro avg       0.79      0.65      0.67     11682
weighted avg       0.81      0.82      0.79     11682



In [13]:
cat.score(X_train_f, y_train_f)

np.float64(0.8838548131661174)

In [14]:
from sklearn.metrics import roc_auc_score

# 양성 클래스(이직 성공=1)의 확률
y_proba = cat.predict_proba(X_test_f)[:, 1]

auc = roc_auc_score(y_test_f, y_proba)
print(f"AUC-ROC: {auc:.4f}")


AUC-ROC: 0.8297


##Diverse Counterfactual Explanations (DiCE) for ML

In [17]:
%cd /content/drive/MyDrive/2030런닝중 - 정형프로젝트/Project/ryui hwang

/content/drive/.shortcut-targets-by-id/1nyTaoNXGEUbaxGyohoBtcBoyRV-posbk/2030런닝중 - 정형프로젝트/Project/ryui hwang


In [19]:
!pip install -U pip setuptools wheel
!pip install -r requirements-dev.txt
!pip install -e .


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 20.1 MB/s  0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
ERROR: Exception:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 107, in _run_wrapper
    status = _inner_run()
             ^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 98, in _inner_run
    return self.run(options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 85, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^

In [18]:
!pwd

/content/drive/.shortcut-targets-by-id/1nyTaoNXGEUbaxGyohoBtcBoyRV-posbk/2030런닝중 - 정형프로젝트/Project/ryui hwang


In [20]:
cat.save_model("cat_model.cbm")

In [ ]:
%cd DiCE

In [24]:
!pip install dice-ml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 8.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [dice-ml]


In [29]:
!pip install -e .

Obtaining file:///content/drive/.shortcut-targets-by-id/1nyTaoNXGEUbaxGyohoBtcBoyRV-posbk/2030%E1%84%85%E1%85%A5%E1%86%AB%E1%84%82%E1%85%B5%E1%86%BC%E1%84%8C%E1%85%AE%E1%86%BC%20-%20%E1%84%8C%E1%85%A5%E1%86%BC%E1%84%92%E1%85%A7%E1%86%BC%E1%84%91%E1%85%B3%E1%84%85%E1%85%A9%E1%84%8C%E1%85%A6%E1%86%A8%E1%84%90%E1%85%B3/Project/ryui%20hwang/DiCE
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for dice_ml (pyproject.toml) ... done
  Created wheel for dice_ml: filename=dice_ml-0.12-0.editable-py3-none-any.whl size=10780 sha256=9f01c90de043b79b2629a4e3a60b259f4e883575b873ce808c0b0d75f6ce16b3
  Stored in directory: /tmp/pip-ephem-wheel-cache-km6ykld9/wheels/fb/4a/ab/6fc128c5a7518760ac7d692057724d15f0c399e984153459fb
Successfully built dice_ml
  Attempting uninstall: dice_ml
    Found existing installation: dice_ml 0.

In [30]:
!pip install -r requirements.txt
# Additional dependendies for deep learning models
!pip install -r requirements-deeplearning.txt
# For running unit tests
!pip install -r requirements-test.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 2.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 kB 3.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 8.9 MB/s  0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16/16 [jupyter]


In [31]:
# Sklearn imports
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

# DiCE imports
import dice_ml
from dice_ml.utils import helpers  # helper functions

In [ ]:
datas_final = datas_2030[[
    '성별코드',
    '만연령',
    '연령계층코드',
    '혼인상태코드',
    '가구주관계코드',
    '교육정도_학력구분코드',
    '교육정도_계열코드',
    '가장최근학교_년제수',
    '가장최근학교_졸업및중퇴연월',
    '졸업연도',
    '가장최근학교_휴학경험유무',
    '가장최근학교_휴학기간년수',
    '직업교육및직장체험_직업교육수혜구분코드',
    '직업교육및직장체험_취업관련시험준비여부',
    '직업교육및직장체험_취업관련시험준비분야코드',
    '직업교육및직장체험_직장체험주요형태코드',
    '직업교육및직장체험_재학휴학중직장체험여부',
    '직업교육및직장체험_재학휴학중직장체험기간코드',
    '구직사항_구직활동기간',
    '구직사항_주요구직방법1코드',
    '구직사항_주요구직경로1코드'
    ]]

In [40]:
X = datas_final

datas_2030['이전직장사항_전직유무'] = datas_2030['이전직장사항_전직유무'].replace(2, 0)
y = datas_2030['이전직장사항_전직유무']

# 1) train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)


/tmp/ipython-input-2198377097.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datas_2030['이전직장사항_전직유무'] = datas_2030['이전직장사항_전직유무'].replace(2, 0)


In [41]:
#train, test 데이터 shape 확인하기
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(46726, 21)
(11682, 21)
(46726,)
(11682,)


In [42]:
import dice_ml
from dice_ml import Dice

# (필수) outcome 컬럼명
OUTCOME_COL = '이전직장사항_전직유무'

# (선택) 숫자코드지만 범주형인 컬럼이 있다면 여기에 추가 (예: 산업코드, 직군코드 등)
NUMERIC_CODED_CATEGORICAL = [
    # "현재일관련사항_7차직업대분류코드",
    # "현재일관련사항_10차산업대분류코드",
]

# (필수) 절대 바꾸면 안 되는 컬럼(immutable) 예시
IMMUTABLE_FEATURES = [
    '성별코드',
    '만연령',
    '연령계층코드',
    '혼인상태코드',
    '가구주관계코드',
    '교육정도_학력구분코드',
    '교육정도_계열코드',
    '가장최근학교_년제수',
    '가장최근학교_졸업및중퇴연월',
    '졸업연도',
    '가장최근학교_휴학경험유무',
    '가장최근학교_휴학기간년수',
    '직업교육및직장체험_직업교육수혜구분코드'
]

# (필수) 바꿀 수 있는 컬럼(actionable) 예시: immutable 제외한 것 중 현실적으로 가능한 것만
ACTIONABLE_FEATURES = [
    '직업교육및직장체험_취업관련시험준비여부',
    '직업교육및직장체험_취업관련시험준비분야코드',
    '직업교육및직장체험_직장체험주요형태코드',
    '직업교육및직장체험_재학휴학중직장체험여부',
    '직업교육및직장체험_재학휴학중직장체험기간코드',
    '구직사항_구직활동기간',
    '구직사항_주요구직방법1코드',
    '구직사항_주요구직경로1코드'
]


In [43]:
X_train

,성별코드,만연령,연령계층코드,혼인상태코드,가구주관계코드,교육정도_학력구분코드,교육정도_계열코드,가장최근학교_년제수,가장최근학교_졸업및중퇴연월,졸업연도,...,가장최근학교_휴학기간년수,직업교육및직장체험_직업교육수혜구분코드,직업교육및직장체험_취업관련시험준비여부,직업교육및직장체험_취업관련시험준비분야코드,직업교육및직장체험_직장체험주요형태코드,직업교육및직장체험_재학휴학중직장체험여부,직업교육및직장체험_재학휴학중직장체험기간코드,구직사항_구직활동기간,구직사항_주요구직방법1코드,구직사항_주요구직경로1코드
13777,1,36,5,1,1,3,1,0,0,2004,...,0,0,0,0,0,0,0,0,0,0
21806,2,30,4,2,1,4,22,2,201402,2014,...,1,3,2,0,2,1,6,0,0,0
15192,1,29,3,1,1,4,100,2,201402,2014,...,2,1,2,0,0,2,0,0,0,0
57322,2,28,3,1,3,3,1,3,201102,2011,...,0,1,2,0,0,2,0,0,0,0
21965,2,30,4,1,1,5,10,4,201302,2013,...,0,1,2,0,0,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40024,2,30,4,1,3,5,70,4,201902,2019,...,4,1,2,0,2,1,5,0,0,0
13637,1,38,5,2,1,4,70,0,0,2006,...,0,0,0,0,0,0,0,0,0,0
50272,2,30,4,1,1,5,22,4,201902,2019,...,1,1,2,0,2,1,5,0,0,0
14301,1,34,4,2,1,5,50,4,201608,2016,...,2,1,1,7,2,1,3,0,0,0


In [44]:

# -----------------------------
# 1) DiCE Data를 만들기 위한 df 구성: (X + y)
#   - DiCE는 "데이터 설명서"가 필요해서 outcome을 df에 포함시킴
# -----------------------------
def build_dice_data_df(X: pd.DataFrame, y: pd.Series, outcome_col: str) -> pd.DataFrame:
    df = X.copy()
    df[outcome_col] = y.values
    return df

df_train = build_dice_data_df(X_train, y_train, OUTCOME_COL)

# -----------------------------
# 2) categorical / continuous 자동 추정 + 보정
# -----------------------------
def infer_feature_types(X: pd.DataFrame, numeric_coded_categorical=None):
    numeric_coded_categorical = set(numeric_coded_categorical or [])
    categorical = set(X.select_dtypes(include=["object", "category", "bool"]).columns.tolist())
    categorical |= numeric_coded_categorical
    categorical = [c for c in X.columns if c in categorical]
    continuous = [c for c in X.columns if c not in set(categorical)]
    return continuous, categorical

continuous_features, categorical_features = infer_feature_types(
    X_train, numeric_coded_categorical=NUMERIC_CODED_CATEGORICAL
)

# (검증) actionable/immutable이 컬럼에 존재하는지 체크
def _check_cols_exist(cols, X, name):
    missing = [c for c in cols if c not in X.columns]
    if missing:
        raise ValueError(f"{name}에 X에 없는 컬럼이 있어요: {missing}")

_check_cols_exist(IMMUTABLE_FEATURES, X_train, "IMMUTABLE_FEATURES")
_check_cols_exist(ACTIONABLE_FEATURES, X_train, "ACTIONABLE_FEATURES")


In [52]:
ONEHOT_COLS = onehot_cols
OHE_FEATURE_ORDER = X_train_f.columns.tolist()
RAW_FEATURE_ORDER = X_train.columns.tolist()


In [53]:
def raw_to_ohe(df_raw: pd.DataFrame,
               onehot_cols: list,
               ohe_feature_order: list) -> pd.DataFrame:
    df_ohe = pd.get_dummies(df_raw, columns=onehot_cols, drop_first=True)
    df_ohe = df_ohe.reindex(columns=ohe_feature_order, fill_value=0)
    return df_ohe


In [54]:
import numpy as np

class CatBoostRawWrapper:
    def __init__(self, cat_model, onehot_cols, ohe_feature_order, raw_feature_order=None):
        self.cat_model = cat_model
        self.onehot_cols = list(onehot_cols)
        self.ohe_feature_order = list(ohe_feature_order)
        self.raw_feature_order = list(raw_feature_order) if raw_feature_order is not None else None

    def _ensure_df(self, X):
        if isinstance(X, pd.DataFrame):
            df = X.copy()
        else:
            if self.raw_feature_order is None:
                raise ValueError("numpy 입력이 들어왔는데 raw_feature_order가 없어요. DataFrame으로 넘기거나 raw_feature_order를 지정하세요.")
            df = pd.DataFrame(X, columns=self.raw_feature_order)
        if self.raw_feature_order is not None:
            df = df.reindex(columns=self.raw_feature_order)
        return df

    def predict_proba(self, X):
        df_raw = self._ensure_df(X)
        df_ohe = raw_to_ohe(df_raw, self.onehot_cols, self.ohe_feature_order)
        return self.cat_model.predict_proba(df_ohe)

    def predict(self, X):
        proba = self.predict_proba(X)[:, 1]
        return (proba >= 0.5).astype(int)


In [58]:
import dice_ml
from dice_ml import Dice

OUTCOME_COL = "이전직장사항_전직유무"

# DiCE용 raw df_train 생성
df_train = X_train.copy()
df_train[OUTCOME_COL] = y_train.values

# raw space에서 연속/범주 지정 (원-핫 전이므로 원본 기준으로!)
# 네가 이미 정의한 것 활용하면 됨:
continuous_features = numeric_cols  # ['만연령','구직사항_구직활동기간']
categorical_features = [c for c in X_train.columns if c not in continuous_features]

d = dice_ml.Data(
    dataframe=df_train,
    continuous_features=continuous_features,
    categorical_features=categorical_features,
    outcome_name=OUTCOME_COL
)

# 🔥 핵심: raw 입력을 받는 wrapper를 DiCE 모델로 사용
cat_wrapped = CatBoostRawWrapper(
    cat_model=cat,
    onehot_cols=ONEHOT_COLS,
    ohe_feature_order=OHE_FEATURE_ORDER,
    raw_feature_order=RAW_FEATURE_ORDER
)

m = dice_ml.Model(model=cat_wrapped, backend="sklearn", model_type="classifier")
exp = Dice(d, m, method="random")


In [66]:
import numpy as np

ACTIONABLE_FEATURES = [c for c in X_train.columns if c not in IMMUTABLE_FEATURES]

def generate_recourse_for_one(query_row_raw: pd.DataFrame, total_cfs=3, desired_class=1):
    cf = exp.generate_counterfactuals(
        query_instances=query_row_raw,
        total_CFs=total_cfs,
        desired_class=desired_class,
        features_to_vary=ACTIONABLE_FEATURES
    )
    cf.visualize_as_dataframe(show_only_changes=True)
    return cf

# 실패 예측 샘플 하나 뽑기: 여기서도 반드시 raw->ohe로 cat 평가해야 함
proba = cat_wrapped.predict_proba(X_test)[:, 1]
pred = (proba >= 0.5).astype(int)

idx = np.where(pred == 0)[0][0]
query = X_test.iloc[[idx]].copy()
print(idx, proba[idx], pred[idx])

cf = generate_recourse_for_one(query, total_cfs=3, desired_class=1)


1 0.14114469441734373 0


  0%|          | 0/1 [00:00<?, ?it/s]/content/drive/.shortcut-targets-by-id/1nyTaoNXGEUbaxGyohoBtcBoyRV-posbk/2030런닝중 - 정형프로젝트/Project/ryui hwang/DiCE/dice_ml/explainer_interfaces/dice_random.py:116: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  candidate_cfs.at[k, selected_features[k][0]] = random_instances.at[k, selected_features[k][0]]
/content/drive/.shortcut-targets-by-id/1nyTaoNXGEUbaxGyohoBtcBoyRV-posbk/2030런닝중 - 정형프로젝트/Project/ryui hwang/DiCE/dice_ml/explainer_interfaces/dice_random.py:116: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '9' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  candidate_cfs.at[k, selected_features[k][0]] = random_instances.at[k, 

Query instance (original outcome : 0)


,성별코드,만연령,연령계층코드,혼인상태코드,가구주관계코드,교육정도_학력구분코드,교육정도_계열코드,가장최근학교_년제수,가장최근학교_졸업및중퇴연월,졸업연도,...,직업교육및직장체험_직업교육수혜구분코드,직업교육및직장체험_취업관련시험준비여부,직업교육및직장체험_취업관련시험준비분야코드,직업교육및직장체험_직장체험주요형태코드,직업교육및직장체험_재학휴학중직장체험여부,직업교육및직장체험_재학휴학중직장체험기간코드,구직사항_구직활동기간,구직사항_주요구직방법1코드,구직사항_주요구직경로1코드,이전직장사항_전직유무
0,2,29,3,2,2,4,92,2,201302,2013,...,1,2,0,4,1,2,0,0,0,0



Diverse Counterfactual set (new outcome: 1)


,성별코드,만연령,연령계층코드,혼인상태코드,가구주관계코드,교육정도_학력구분코드,교육정도_계열코드,가장최근학교_년제수,가장최근학교_졸업및중퇴연월,졸업연도,...,직업교육및직장체험_직업교육수혜구분코드,직업교육및직장체험_취업관련시험준비여부,직업교육및직장체험_취업관련시험준비분야코드,직업교육및직장체험_직장체험주요형태코드,직업교육및직장체험_재학휴학중직장체험여부,직업교육및직장체험_재학휴학중직장체험기간코드,구직사항_구직활동기간,구직사항_주요구직방법1코드,구직사항_주요구직경로1코드,이전직장사항_전직유무
0,-,-,-,-,-,-,-,-,-,-,...,-,1,-,-,-,4,-,-,-,-
1,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,3,-,-,1
2,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,14,-,5,1
